In [60]:
import numpy as np
import json

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import itertools

In [179]:
from area import area



https://akm-img-a-in.tosshub.com/indiatoday/images/story/201812/Latitude_and_Longitude_of_the_Earth.png?QNj7A2r7Kqx20iTOJl8JAn3Q476b7LJ4&size=770:433

Test input
```
34.026793,-118.488928,blue,square
34.027518,-118.487853,blue,square
34.026189,-118.486335,blue,square
34.025447,-118.487280,blue,square
34.026793,-118.488928,blue,square


34.026794,-118.487506,green,square
34.027830,-118.486390,red,square
```

In [91]:
ext = {
  "path": {
    "Md": [
      {
        "lat": 34.026793,
        "lng": -118.488928
      },
      {
        "lat": 34.027518,
        "lng": -118.487853
      },
      {
        "lat": 34.026189,
        "lng": -118.486335
      },
      {
        "lat": 34.025447,
        "lng": -118.487280
      },
      {
        "lat": 34.027748,
        "lng": -118.489069
      }
    ],
    "gm_accessors_": {
      "length": {"length": 0}
    },
    "length": 4,
    "gm_bindings_": {
      "length": {"length": 0}
    }
  },
  "encodedPath": "ys~nElvdqU`jXhvIfcLeuq@muYc~E",
  "randomValue": 0.5241
}

In [92]:
ext

{'path': {'Md': [{'lat': 34.026793, 'lng': -118.488928},
   {'lat': 34.027518, 'lng': -118.487853},
   {'lat': 34.026189, 'lng': -118.486335},
   {'lat': 34.025447, 'lng': -118.48728},
   {'lat': 34.027748, 'lng': -118.489069}],
  'gm_accessors_': {'length': {'length': 0}},
  'length': 4,
  'gm_bindings_': {'length': {'length': 0}}},
 'encodedPath': 'ys~nElvdqU`jXhvIfcLeuq@muYc~E',
 'randomValue': 0.5241}

In [93]:
def get_dict(json_data):
    return json.loads(json_data)


In [94]:
def polygon_vertices(data):
    coords = data['path']['Md']
    coord_list = [list(i.values()) for i in coords]
    return coord_list

In [95]:
poly_verts = polygon_vertices(ext)
poly_verts

[[34.026793, -118.488928],
 [34.027518, -118.487853],
 [34.026189, -118.486335],
 [34.025447, -118.48728],
 [34.027748, -118.489069]]

In [96]:
def rect_corners(vertices):
    '''get the square around the inputted polygon vertices. only works for NW quadrant e.g. United States'''
    north = max([i[0] for i in vertices])
    south = min([i[0] for i in vertices])
    east = max([i[1] for i in vertices])
    west = min([i[1] for i in vertices])
    
    extrema = [[north,south], [east,west]]
    corners = sorted(list(itertools.product(*extrema)), key = lambda x :(x[0], x[1]))
    square_verts = {'sw': corners[0], 'se': corners[1],'nw':corners[2],'ne':corners[3]}
    return(square_verts)

In [97]:
square_verts = rect_corners(poly_verts)
square_verts

{'sw': (34.025447, -118.489069),
 'se': (34.025447, -118.486335),
 'nw': (34.027748, -118.489069),
 'ne': (34.027748, -118.486335)}

In [90]:
def get_coord_list(coord_dict, dim):
    se = coord_dict['se']
    sw = coord_dict['sw']
    nw = coord_dict['nw']
    ne = coord_dict['ne']
    
    delta_long = (se[1]- sw[1])/(dim-1)
    long_coords = [sw[1]]
    start = sw[1]
    for i in range(dim-1):
        start += delta_long
        long_coords.append(start)
        
    delta_lat = (nw[0] - sw[0])/(dim-1)
    
    lat_coords = [sw[0]]
    start = sw[0]
    for i in range(dim-1):
        start += delta_lat
        lat_coords.append(start)
        
    coords = [lat_coords, long_coords]
    coord_combos = list(itertools.product(*coords))
    
    return coord_combos

In [130]:
get_coord_list(square_verts, 5)

[(34.025447, -118.489069),
 (34.025447, -118.48838549999999),
 (34.025447, -118.48770199999998),
 (34.025447, -118.48701849999998),
 (34.025447, -118.48633499999997),
 (34.02602225, -118.489069),
 (34.02602225, -118.48838549999999),
 (34.02602225, -118.48770199999998),
 (34.02602225, -118.48701849999998),
 (34.02602225, -118.48633499999997),
 (34.026597499999994, -118.489069),
 (34.026597499999994, -118.48838549999999),
 (34.026597499999994, -118.48770199999998),
 (34.026597499999994, -118.48701849999998),
 (34.026597499999994, -118.48633499999997),
 (34.02717274999999, -118.489069),
 (34.02717274999999, -118.48838549999999),
 (34.02717274999999, -118.48770199999998),
 (34.02717274999999, -118.48701849999998),
 (34.02717274999999, -118.48633499999997),
 (34.02774799999999, -118.489069),
 (34.02774799999999, -118.48838549999999),
 (34.02774799999999, -118.48770199999998),
 (34.02774799999999, -118.48701849999998),
 (34.02774799999999, -118.48633499999997)]

In [136]:
def within(point, bounds):
    '''pass in a single point and a dictionary of corners (se, sw, ne, nw)'''
    bounds = list(bounds.values())
    point = Point(point)
    polygon = Polygon(bounds)
    return polygon.contains(point)

In [172]:
def get_sample(json_data):
#     data = json.loads(json_data)
    data = json_data #need to swap this with the line above in the real implementation
    poly_verts = polygon_vertices(data)
    square_verts = rect_corners(poly_verts)
    lst = np.array(get_coord_list(square_verts, 100))
    print(len(lst))
    t = np.array([within(i, square_verts) for i in lst])
    return str(t)

In [174]:
get_sample(ext)

10000


'[False False False ...  True  True False]'

In [173]:
np.random.choice(np.arange(len(get_sample(ext))), 5, replace = False)

10000


array([40, 24,  4, 26, 30])

In [181]:
ext2 = {
  "path": {
    "Md": [
      {
        "lat": 34.2206,
        "lng": -118.5448
      },
      {
        "lat": 34.2207,
        "lng": -118.5361
      },
      {
        "lat": 34.2158,
        "lng": -118.5359
      },
      {
        "lat": 34.2138,
        "lng": -118.5408
      },
      {
        "lat": 34.2157,
        "lng": -118.5448
      }
    ],
    "gm_accessors_": {
      "length": {}
    },
    "length": 5,
    "gm_bindings_": {
      "length": {}
    }
  },
  "encodedPath": "}uzoE|gprUMeu@z]a@jKp]wJ|W",
  "randomValue": 0.6121
}

In [177]:
def get_area(json_data):
    '''
    returns the area of the coordinate space in the target json file

    in: json data with coordinates under ['path']['Md'] item
    out: area of polygon defined in json data (square meters), plus number of vertices

    router extension: /polygon
    e.g. base_url/polygon/{data}
    '''

    coords = json_data['path']['Md']

    coord_list = [[list(i.values()) for i in coords]]
    coord_list[0].append(coord_list[0][0])

    #calculate and return area
    obj = {'type':'Polygon','coordinates':coord_list}
    sqm = area(obj)
    return sqm

In [182]:
get_area(ext2)

306600.47719959053